## Sentence Transformers (specifically stsb-mpnet-base-v2)
This is THE BEST sentence-level embedding model on huggingface. 
But we'll see if it's good enough for the real world. 


In [ ]:
!pip install sentence-transformers

In [11]:
from sentence_transformers import SentenceTransformer
sentences = ["This is an example sentence", "Each sentence is converted", "Kastan is a fun programmer"]

model = SentenceTransformer('sentence-transformers/stsb-mpnet-base-v2')
embeddings = model.encode(sentences)
# print(embeddings)
print("embeddings.shape:", embeddings.shape)

score01 = embeddings[0] @ embeddings[1] #1.0473
score02 = embeddings[0] @ embeddings[2] #1.0095
# score02 = embeddings[0] @ embeddings[3] #1.0095

print(score01, score02) # the first two are closer than the first and third

embeddings.shape: (3, 768)
5.53563 0.17808735


# Doc Query
This does pure question to text lookup (no generation).
But I like that because hopefully it's more factual. 

Also this implementation works directly with PDFs! That's awesome for easily using all kinds of new data!

In [ ]:
!pip install docquery

In [3]:
from docquery import document, pipeline
p = pipeline('document-question-answering')

# using shorter PDF for quick testing
doc = document.load_document("../data-generator/notes/Student_Notes_short.pdf")

# use full pdf for real testing
# doc = document.load_document("../data-generator/notes/Student Notes.pdf")

In [4]:
questions = [
  "What are boolean logic operations?", 
  "What is overflow?",
  ]

for q in questions:
  print(q, p(question=q, **doc.context))

huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
huggingface/tokenizers: The current process just got forked, after parallelism has already been used. Disabling parallelism to avoid deadlocks...
To disable this warning, you can either:
	- Avoid using `tokenizers` before the fork if possible
	- Explicitly set the environment variable TOKENIZERS_PARALLELISM=(true | false)
What are boolean logic opera